####  Colab 환경에서 검증이 가능할 수 있도록 작성하였습니다. 
####  /content/drive/MyDrive에 Final_FertiMind_dacon 폴더를 위치시키면 실행 가능합니다!
#### Optuna, catboost 라이브러리 설치 필요합니다.

In [ ]:
from google import colab
colab.drive.mount("/content/drive")

In [ ]:
%cd /content/drive/MyDrive/Final_FertiMind_dacon/
#현재 코드 파일의 경로

In [ ]:
!pip install catboost
!pip install optuna

In [ ]:
import random
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier
import lightgbm as lgb
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

In [ ]:
train = pd.read_csv('./data/train.csv').drop(columns=['ID'])
test = pd.read_csv('./data/test.csv').drop(columns=['ID'])
train.shape, test.shape

### 이상치 제거 + 중요 시술 당시 나이 결측값 보간
train.loc[train['시술 당시 나이'] == '알 수 없음', '시술 당시 나이'] = '만43-44세'
test.loc[test['시술 당시 나이'] == '알 수 없음', '시술 당시 나이'] = '만43-44세'
train = train[~((train["이식된 배아 수"] == 0) & (train["임신 성공 여부"] == 1))]

In [ ]:
### SHapley value 값이 현저히 낮은 열 제거 1
drop_columns =['불임 원인 - 여성 요인', '불임 원인 - 자궁경부 문제', '불임 원인 - 정자 면역학적 요인', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '난자 채취 경과일', '난자 해동 경과일']
train.drop(columns=drop_columns, inplace=True)
test.drop(columns=drop_columns, inplace=True)

train.shape, test.shape

In [ ]:
### 결측 값 보간

nan_col_list = ['착상 전 유전 검사 사용 여부','PGD 시술 여부', 'PGS 시술 여부']
for col in nan_col_list:
    train[col] = train[col].fillna(0)
    test[col] = test[col].fillna(0)

nan_col_list = ['착상 전 유전 진단 사용 여부' ]

for col in nan_col_list:
    train[col] = train[col].fillna(1)
    test[col] = test[col].fillna(1)

In [ ]:
### SHapley value 값이 현저히 낮은 열 제거 2
drop_col = ["불임 원인 - 정자 농도","DI 출산 횟수","DI 임신 횟수","착상 전 유전 진단 사용 여부","난자 혼합 경과일","부부 주 불임 원인","남성 주 불임 원인","저장된 신선 난자 수","대리모 여부","남성 부 불임 원인","여성 부 불임 원인","부부 부 불임 원인","미세 주입에서 생성된 배아 수","여성 부 불임 원인","불명확 불임 원인","불임 원인 - 자궁내막증" ]

for col in train.columns:
    if col in drop_col:
        train.drop(columns=col, inplace=True)
        test.drop(columns=col, inplace=True)

train.shape, test.shape

In [ ]:
### 배아 이식 경과일 column의 결측 값들이 비무작위 결측임으로 판단해 새로운 카테고리로 변환

train['배아 이식 경과일'] = train['배아 이식 경과일'].replace(-1, np.nan)
test['배아 이식 경과일'] = test['배아 이식 경과일'].replace(-1, np.nan)

mapping_dict = {
    0.0: '0일',
    1.0: '1일',
    2.0: '2일',
    3.0: '3일',
    4.0: '4일',
    5.0: '5일',
    6.0: '6일',
    7.0: '7일'
}

train['배아 이식 경과일'] = train['배아 이식 경과일'].map(mapping_dict)
# 3) NaN 값(Missing)을 'Missing' 범주로 대체
train['배아 이식 경과일'] = train['배아 이식 경과일'].fillna('Missing')
train['배아 이식 경과일'] = train['배아 이식 경과일'].astype('category')

test['배아 이식 경과일'] = test['배아 이식 경과일'].map(mapping_dict)
# 3) NaN 값(Missing)을 'Missing' 범주로 대체
test['배아 이식 경과일'] = test['배아 이식 경과일'].fillna('Missing')
test['배아 이식 경과일'] = test['배아 이식 경과일'].astype('category')



In [ ]:
### Key columns에 대한 추가 bin column 생성
if '총 생성 배아 수' in train.columns:
    bins = [-0.1, 0, 5, 10, 15, 20, 999]
    labels = ['0','1~5','6~10','11~15','16~20','21이상']
    train['총_생성_배아_bin'] = pd.cut(train['총 생성 배아 수'], bins=bins, labels=labels)
    test['총_생성_배아_bin']  = pd.cut(test['총 생성 배아 수'],  bins=bins, labels=labels)

if '저장된 배아 수' in train.columns:
    bins = [-0.1, 0, 5, 10, 15, 20, 999]
    labels = ['0','1~5','6~10','11~15','16~20','21이상']
    train['저장된 배아 수_bin'] = pd.cut(train['저장된 배아 수'], bins=bins, labels=labels)
    test['저장된 배아 수_bin']  = pd.cut(test['저장된 배아 수'],  bins=bins, labels=labels)

# 6. 희소 범주(rare categories) 병합
def merge_rare_categories(series, min_count=100):
    """
    시리즈 내에서 출현 횟수가 min_count 미만인 카테고리를
    모두 '기타' 카테고리로 통합
    """
    value_counts = series.value_counts(dropna=False)
    rare_cats = value_counts[value_counts < min_count].index
    return series.replace(rare_cats, '기타')


rare_merge_cols = ['특정 시술 유형', '배아 생성 주요 이유']
for col in rare_merge_cols:
    if col in train.columns:
        train[col] = merge_rare_categories(train[col], min_count=30)
    if col in test.columns:
        test[col] = merge_rare_categories(test[col], min_count=30)
train.drop(columns =['임신 시도 또는 마지막 임신 경과 연수'],inplace = True )
test.drop(columns =['임신 시도 또는 마지막 임신 경과 연수'],inplace = True )


In [ ]:
### 특정 시술 유형 통합

procedure_encoding_dict = {'ICSI' : 'ICSI', 'IVF' : 'IVF', 'Unknown' : 'Unknown','IUI' : 'IUI',
                           'ICSI:ICSI' : 'ICSI', 'ICSI / BLASTOCYST ' : 'ICSI / BLASTOCYST',
                           'IVF / BLASTOCYST' : 'IVF / BLASTOCYST','IVF:IVF' : 'IVF','ICSI:IVF' : 'ICSI:IVF',
                           'ICSI / AH' : 'ICSI / AH','IVF:ICSI' : 'ICSI:IVF', 'IVF / AH' : 'IVF / AH',
                           'ICSI:Unknown' : 'ICSI', 'IVF:Unknown' : 'IVF', 'ICI' : 'ICI','Generic DI' : 'Generic DI',
                           'IVI' : 'IVI','ICSI / BLASTOCYST:IVF / BLASTOCYST' : 'ICSI / BLASTOCYST:IVF / BLASTOCYST',
                           'ICSI / BLASTOCYST :IVF / BLASTOCYST': 'ICSI / BLASTOCYST:IVF / BLASTOCYST','FER' : 'FER',
                           'IVF / AH:ICSI / AH' : 'IVF / AH:ICSI / AH','GIFT':'GIFT','ICSI / AH:Unknown' : 'ICSI / AH',
                           'ICSI / BLASTOCYST :ICSI' : 'ICSI / BLASTOCYST', 'IUI:ICI' : 'IUI:ICI',
                           'IVF:Unknown:Unknown:Unknown' : 'IVF'}

train['특정 시술 유형'] = train['특정 시술 유형'].map(procedure_encoding_dict)
test['특정 시술 유형'] = test['특정 시술 유형'].map(procedure_encoding_dict)

In [ ]:
### 나머지 열들에 대한 결측값 보간
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부",
    '시술 정자',
    '시술 난자',
        "저장된 신선 난자 수","총_생성_배아_bin","저장된 배아 수_bin"
]
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    # "배아 이식 경과일",
    "배아 해동 경과일"
]+['총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수',
                      '총 임신 횟수', 'IVF 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수']

for col in categorical_columns:
  if col in train.columns:
    if train[col].isna().sum() > 0 or test[col].isna().sum() > 0 :

      mode_value = train[col].mode()[0]
      train[col].fillna(mode_value, inplace=True)  # 최빈값으로 대체
      test[col].fillna(mode_value, inplace = True)
    train[col] = train[col].astype(str)
    test[col] = test[col].astype(str)

for col in numeric_columns:
  if col in train.columns:
    if train[col].isna().sum():
      if col == "이식된 배아 수":
        train[col].fillna(3, inplace=True)  
        test[col].fillna(3, inplace = True)
      elif train[col].isna().sum() >= len(train)*0.5: # 결측치가 절반이 넘어갈 경우
        train[col].fillna(-1, inplace=True)  # -1로 대체
        test[col].fillna(-1, inplace = True)
      else:
        mode_value = train[col].mode()[0]  # 최빈값 구하기
        train[col].fillna(mode_value, inplace=True)  # 최빈값으로 대체
        test[col].fillna(mode_value, inplace = True)

In [ ]:
### 파생 변수 생성


def make_feature(df):

  df['미세주입 배아 생성률'] = (df['미세주입에서 생성된 배아 수']) / (df['미세주입된 난자 수'] + 1)
  df['미세주입 배아 이식률'] = (df['미세주입 배아 이식 수']) / (df['미세주입에서 생성된 배아 수'] + 1)
  df['배아 이식률'] = (df['이식된 배아 수']) / (df['총 생성 배아 수'] + 1)
  df['저장 배아 비율'] = (df['저장된 배아 수'])/ (df['총 생성 배아 수'] + 1)
  df['해동 배아 생존율'] = (df['해동된 배아 수'])/ (df['총 생성 배아 수'] + 1)
  df['파트너 정자 혼합 비율'] = (df['파트너 정자와 혼합된 난자 수']) / (df['혼합된 난자 수'] + 1)
  return df

In [ ]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

X = make_feature(X)
test = make_feature(test)

X.shape, test.shape

In [ ]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부",
    '시술 정자',
    '시술 난자',
        "저장된 신선 난자 수","총_생성_배아_bin","임신시도_bin","저장된 배아 수_bin","배아 이식 경과일"
]

categorical_columns_new = []
for col in X.columns:
  if col in categorical_columns:
    categorical_columns_new.append(col)


In [ ]:
from sklearn.utils.class_weight import compute_class_weight

seed = 4972

def objective_cat(trial):

    ##Catboost를 Optuna로 하이퍼파라미터 탐색하는 Objective 함수.
    global best_val_auc
    cbrm_param = {
        'iterations':trial.suggest_int("iterations", 1500, 3000),
        'depth':trial.suggest_int("depth", 4, 10),
        'learning_rate' : trial.suggest_float('learning_rate',0.005, 0.1),
        'l2_leaf_reg': trial.suggest_float("l2_leaf_reg", 1e-4, 100.0, log=True),
        'early_stopping_rounds': 300
    }

    # Generate model
    model_cat = CatBoostClassifier(**cbrm_param, auto_class_weights='Balanced', eval_metric='AUC', random_state=seed, task_type="GPU")
    model_cat.fit(X_train,
          y_train,
          cat_features=categorical_columns_new,
          early_stopping_rounds= 300,
          eval_set=[(X_val,y_val)],
          verbose = 300)

    val_auc = model_cat.get_best_score()["validation"]["AUC"]
    best_model_path = "./weight/cat_fold_gpu{}.cbm".format(idx)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        model_cat.save_model(best_model_path)
        print('저장 완료')

    return val_auc

def objective_lgbm(trial):
    """
    LightGBM을 Optuna로 하이퍼파라미터 탐색하는 Objective 함수.
    """
    lgbm_params = {
        "objective": trial.suggest_categorical("objective", ["binary"]),
        "metric": trial.suggest_categorical("metric", ["auc"]),
        "random_state": trial.suggest_int('random_state', 42, 42),
        "verbose": trial.suggest_int('verbose', -1, -1),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50, step=5),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0, step=0.1),
    }

    class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y), y=y_train)
    class_weight_dict = {i: w for i, w in enumerate(class_weights)}

    model_lgbm = lgb.LGBMClassifier(**lgbm_params, class_weight=class_weight_dict)
    model_lgbm.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        categorical_feature=categorical_columns_new
    )

    return model_lgbm.best_score_['valid_0']['auc']

In [ ]:
import pandas as pd
import numpy as np

for i, col in enumerate(categorical_columns_new):
    # 1) NaN(결측값) 여부 확인
    nan_mask = train[col].isnull()
    if nan_mask.any():
        print(f"[열 인덱스: {i}] {col} 열에 NaN(결측값) 존재")
        print(train.loc[nan_mask, col].head(), "\n")

    # 2) int/str 이외의 타입이 있는지 확인 (float, bool 등)
    invalid_mask = train[col].notnull() & ~train[col].apply(lambda x: isinstance(x, (int, str)))
    if invalid_mask.any():
        print(f"[열 인덱스: {i}] {col} 열에 int/str 이외의 타입 존재")
        print(train.loc[invalid_mask, col].head(), "\n")


# 재현성 확인 코드

###### 재구현을 위한 Weight및 파라미터를 불러오는 코드입니다. Training 코드는 해당 재구현 코드 밑에 구현되어있습니다.
###### Optuna 검색을 두 번 실행하여, 각 실행에서 CatBoost 모델 10개와 LGBM 모델 10개씩, 총 40개 모델을 생성하였습니다.
###### 이 중 검증 성능이 가장 낮은 두 개의 fold에 해당하는 8개의 모델을 제외하고, 나머지 32개 모델에 대해 앙상블을 진행하였습니다.

In [ ]:
import json
# with open("lgbm_params_minsu_trial_2.json", "w") as f:
#     json.dump(lgbm_param_list, f)

with open("./lgbm_parameter/lgbm_params_minsu_trial_180_1.json", "r") as f:
    loaded_lgbm_params_60 =  json.load(f)

# import json
# with open("lgbm_params_minsu_trial.json", "w") as f:
#     json.dump(lgbm_param_list, f)

with open("./lgbm_parameter/lgbm_params_minsu_trial_200_1.json", "r") as f:
    loaded_lgbm_params_100 =  json.load(f)
for col in X.columns:
    if X[col].dtype == 'object':  # 문자형이면 category로 변환
        X[col] = X[col].astype('category')
        test[col] = test[col].astype('category')
seed = 4972

test_pred_total = []
scores_total = []
seed_everything(seed)
print('Seed {} Start!'.format(seed))
cv = StratifiedKFold(n_splits = 10, random_state = seed, shuffle=True)

lgbm_param_list = []
for idx, (train_idx, val_idx) in enumerate(cv.split(X, y)):
    print('='*50)
    X_train, X_val = X.iloc[train_idx],X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    best_val_auc = 0
    ### 10개의 Fold 중에 검증 성능이 가장 낮은 두개의 Fold를 제외하여서 모델을 구성 Fold1 and 9
    if idx == 1:
        continue
    if idx == 9:
        continue


    ### 첫번째 Catboost 모델 불러오기
    model_gpu_60 = CatBoostClassifier()
    model_gpu_60.load_model("./weight_now_best_solo_first/cat_fold_gpu{}.cbm".format(idx))

    lgbm_trial_params = loaded_lgbm_params_60[idx]
    class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y), y=y_train)
    class_weight_dict = {i: w for i, w in enumerate(class_weights)}
    ### 첫번쨰 Lgbm 모델 불러오기
    model_lgbm_60 = lgb.LGBMClassifier(**lgbm_trial_params,class_weight=class_weight_dict)
    model_lgbm_60.fit(X_train, y_train, eval_set=[(X_val, y_val)],categorical_feature=categorical_columns_new)


    val_pred_gpu_60 = model_gpu_60.predict_proba(X_val)[:, 1]
    val_pred_lgbm_60 = model_lgbm_60.predict_proba(X_val)[:, 1]

    ### 두번째 Catboost 모델 불러오기
    model_gpu = CatBoostClassifier()
    model_gpu.load_model("./weight_now_best_for_ensemble/cat_fold_gpu{}.cbm".format(idx))

    lgbm_trial_params = loaded_lgbm_params_100[idx]
    class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y), y=y_train)
    class_weight_dict = {i: w for i, w in enumerate(class_weights)}
    ### 두번째 Lgbm 모델 불러오기
    model_lgbm = lgb.LGBMClassifier(**lgbm_trial_params,class_weight=class_weight_dict)
    model_lgbm.fit(X_train, y_train, eval_set=[(X_val, y_val)],categorical_feature=categorical_columns_new)


    val_pred_gpu = model_gpu.predict_proba(X_val)[:, 1]
    val_pred_lgbm = model_lgbm.predict_proba(X_val)[:, 1]

    print('Cat val score : {}'.format(roc_auc_score(y_val,val_pred_gpu_60)))
    print('Cat gpu val score : {}'.format(roc_auc_score(y_val,val_pred_gpu)))
    print('LGBM val score : {}'.format(roc_auc_score(y_val,val_pred_lgbm_60)))
    print('LGBM val score : {}'.format(roc_auc_score(y_val,val_pred_lgbm)))

    roc_val = roc_auc_score(y_val, ((val_pred_gpu+val_pred_gpu_60)*1.8 + val_pred_lgbm_60+val_pred_lgbm)/5.6)
    scores_total.append(roc_val)
    print('Ensemble val score : {}'.format(roc_val))


    test_pred_gpu_60 = model_gpu_60.predict_proba(test)[:, 1]
    test_pred_lgbm_60 = model_lgbm_60.predict_proba(test)[:, 1]

    test_pred_gpu = model_gpu.predict_proba(test)[:, 1]
    test_pred_lgbm = model_lgbm.predict_proba(test)[:, 1]

    ### Catboost Lgbm 1.8대 1의 앙상블 진행
    test_pred_total.append(((test_pred_gpu+test_pred_gpu_60)*1.8+test_pred_lgbm+test_pred_lgbm_60)/5.6)


In [ ]:
np.mean(scores_total), scores_total
test_pred_total_mean = np.mean(test_pred_total, axis=(0))
sample_submission = pd.read_csv('./data/sample_submission.csv')
sample_submission['probability'] = test_pred_total_mean
sample_submission.to_csv('Add__Minsu_trial_cat1.8lgbm1_fold8.csv', index=False)

# Optuna Training 코드

In [ ]:

for col in X.columns:
    if X[col].dtype == 'object':  # 문자형이면 category로 변환
        X[col] = X[col].astype('category')
        test[col] = test[col].astype('category')
seed = 4972

test_pred_total = []
scores_total = []
seed_everything(seed)
print('Seed {} Start!'.format(seed))
cv = StratifiedKFold(n_splits = 10, random_state = seed, shuffle=True)
cat_param_list = []
lgbm_param_list = []
for idx, (train_idx, val_idx) in enumerate(cv.split(X, y)):
    print('='*50)
    X_train, X_val = X.iloc[train_idx],X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    best_val_auc = 0

    sampler = TPESampler(seed=seed)

    #### Catboost optuna로 최적 파라미터 찾기
    optuna_cat = optuna.create_study(direction='maximize', sampler=sampler)
    optuna_cat.optimize(objective_cat, n_trials = 30)
    cat_trial = optuna_cat.best_trial
    cat_trial_params = cat_trial.params
    cat_param_list.append(cat_trial_params)

    model = CatBoostClassifier(**cat_trial_params, auto_class_weights='Balanced', eval_metric='AUC', random_state=seed, task_type="GPU")
    model.load_model("./weight/cat_fold_gpu{}.cbm".format(idx))


    sampler = TPESampler(seed=seed)

    #### Lgbm optuna로 최적 파라미터 찾기
    optuna_lgbm = optuna.create_study(direction='maximize', sampler=sampler)
    optuna_lgbm.optimize(objective_lgbm, n_trials=30)
    lgbm_trial = optuna_lgbm.best_trial
    lgbm_trial_params = lgbm_trial.params
    lgbm_param_list.append(lgbm_trial_params)

    class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y), y=y_train)
    class_weight_dict = {i: w for i, w in enumerate(class_weights)}
    model_lgbm = lgb.LGBMClassifier(**lgbm_trial_params, class_weight=class_weight_dict)
    model_lgbm.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        categorical_feature=categorical_columns_new)
    val_pred = model.predict_proba(X_val)[:, 1]
    val_pred_lgbm = model_lgbm.predict_proba(X_val)[:, 1]
    print('Cat val score : {}'.format(roc_auc_score(y_val,val_pred)))
    print('LGBM val score : {}'.format(roc_auc_score(y_val,val_pred_lgbm)))


    ### Catboost Lgbm 1.8대 1의 앙상블 진행
    roc_val = roc_auc_score(y_val, (1.8*val_pred+val_pred_lgbm)/2.8)
    scores_total.append(roc_val)
    print('Ensemble val score : {}'.format(roc_val))
    test_pred = model.predict_proba(test)[:, 1]
    test_pred_lgbm = model_lgbm.predict_proba(test)[:, 1]
    test_pred_total.append((1.8*test_pred+test_pred_lgbm)/2.8)


In [ ]:
np.mean(scores_total), scores_total

In [ ]:
np.mean(scores_total), scores_total
test_pred_total_mean = np.mean(test_pred_total, axis=(0))
sample_submission = pd.read_csv('./data/sample_submission.csv')
sample_submission['probability'] = test_pred_total_mean
sample_submission.to_csv('Add__Minsu_trial_cat1.8lgbm1_fold8.csv', index=False)